In [1]:
""" 
g = -10 
GPS = [xt,yt,zt]
base station = [D1,D2,D3,D4] from z=10 
IMU = [xt,yt,zt,vxt,vyt,vzt] 
motion noise = N(0,R) ; R is diag(sigx^2,sigy^2,sigz^2,sigvx^2,sigvy^2,sigvz^2) 
sigx=sigy=sigz=0.01
sigvx=sigvy=sigvz=0.1 

measurement noise = N(0,Q) Q = sig^2I 
sigGPS = 0.1 
sigBS = 0.1 
sigIMU = 0.1 

ut is only g in zt direction 

Time in seconds 
N = 130 time steps , delta_time = 0.01 
field is 100 x 64 , 
goal post is 8 x 3 , at (4,50,0);(4,50,3);(-4,50,3);(-4,50,0); 

boal init = [24.0,4.0,0.0,−16.04,36.8,8.61]T , with sigma = 1e-4 I_6 
state Xt = [xt,yt,zt,vxt,vyt,vzt]T 
"""
import numpy as np 
import math
import scipy.stats as stats 
import pandas as pd
from numpy.random import multivariate_normal  
import plotly.express as px
import plotly.graph_objects as go
import plotly.subplots as sp  
plot_dir = "plots_part_2/"
tol = 0

In [2]:
g = -10 
def u(t):
    return np.array([g]).reshape(1,1) 
time_step = 0.01 
N = 130 
time = [i*time_step for i in range(N)]


In [3]:
# Part (a) motion model and Part (b) observation model 
sigGPS = 0.1 
sigBS = 0.1 
sigIMU = 0.1  
sigx=sigy=sigz=0.01 
sigvx=sigvy=sigvz=0.1

BS_height =10 
D1_pos = np.transpose(np.array([-32,50,BS_height])).reshape((3,1)) 
D2_pos = np.transpose(np.array([32,50,BS_height])).reshape((3,1))
D3_pos = np.transpose(np.array([32,-50,BS_height])).reshape((3,1))
D4_pos = np.transpose(np.array([-32,-50,BS_height])).reshape((3,1))

R = np.diag([sigx**2,sigy**2,sigz**2,sigvx**2,sigvy**2,sigvz**2])
Q_GPS = sigGPS**2*np.eye(3) 
Q_BS = sigBS**2*np.eye(4) 
Q_IMU = sigIMU**2*np.eye(6) 

mu_0 = np.transpose(np.array([24.0,4.0,0.0,-16.04,36.8,8.61]))
mu_0 = mu_0.reshape((6,1)) 
sigma_0 = 1e-4*np.eye(6) 

A_t = np.array([
    [1,0,0,time_step,0,0],
    [0,1,0,0,time_step,0],
    [0,0,1,0,0,time_step],
    [0,0,0,1,0,0],
    [0,0,0,0,1,0],
    [0,0,0,0,0,1]
    ])
B_t =  np.transpose(np.array([0,0,0.5*(time_step**2),0,0,time_step])) 
B_t = B_t.reshape((6,1))

def initial_state(mu_0,sigma_0):
    state_0 = multivariate_normal(mu_0.reshape(6),sigma_0).reshape((6,1)) 
    return state_0 

def motion_model(mu_prev,u_t,R=R):
    return A_t@mu_prev + B_t@u_t + multivariate_normal(np.zeros(6),R).reshape((6,1))

Ct_GPS = np.array([
    [1,0,0,0,0,0],
    [0,1,0,0,0,0],
    [0,0,1,0,0,0]
])
Ct_IMU = np.eye(6) 
 
def Ht(xt):
    D1 = np.linalg.norm(D1_pos - xt[0:3]).item()
    D2 = np.linalg.norm(D2_pos - xt[0:3]).item()
    D3 = np.linalg.norm(D3_pos - xt[0:3]).item()
    D4 = np.linalg.norm(D4_pos - xt[0:3]).item()

    return np.array([D1,D2,D3,D4]).reshape(4,1) 

def JacHt(xt):
    D1 = np.linalg.norm(D1_pos - xt[0:3]).item()
    D2 = np.linalg.norm(D2_pos - xt[0:3]).item()
    D3 = np.linalg.norm(D3_pos - xt[0:3]).item()
    D4 = np.linalg.norm(D4_pos - xt[0:3]).item()

    J = np.zeros((4,6))

    J[0,0] = (xt[0]-D1_pos[0]).item()/D1
    J[0,1] = (xt[1]-D1_pos[1]).item()/D1
    J[0,2] = (xt[2]-D1_pos[2]).item()/D1
    J[1,0] = (xt[0]-D2_pos[0]).item()/D2
    J[1,1] = (xt[1]-D2_pos[1]).item()/D2
    J[1,2] = (xt[2]-D2_pos[2]).item()/D2
    J[2,0] = (xt[0]-D3_pos[0]).item()/D3
    J[2,1] = (xt[1]-D3_pos[1]).item()/D3
    J[2,2] = (xt[2]-D3_pos[2]).item()/D3
    J[3,0] = (xt[0]-D4_pos[0]).item()/D4
    J[3,1] = (xt[1]-D4_pos[1]).item()/D4
    J[3,2] = (xt[2]-D4_pos[2]).item()/D4
    return J

def extended_kalman_filter(mu_prev,sigma_prev,u_t,z_t=None,R=None,C_t=None,Q=None,extended=False):
    mu_t = A_t@mu_prev + B_t@u_t 
    sigma_t = A_t@sigma_prev@np.transpose(A_t) + R 
    K_t = sigma_t@np.transpose(C_t)@np.linalg.inv((C_t@sigma_t@np.transpose(C_t)+Q))
    if z_t is not None:
        estimate_zt = C_t@mu_t if not extended else Ht(mu_t) 
        mu_t = mu_t + K_t@(z_t-estimate_zt)
        # sigma_t = (np.array([[1,0],[0,1]])-K_t@C_t)@sigma_t
        sigma_t = (np.eye(6) - K_t @ C_t) @ sigma_t @ (np.eye(6) - K_t @ C_t).T + K_t @ Q @ K_t.T
    return mu_t,sigma_t,K_t 

def gen_ground_truth_and_measurements(R=R,Q_GPS=Q_GPS,Q_BS=Q_BS,Q_IMU=Q_IMU):
    state_t = [initial_state(mu_0,sigma_0)]
    for i,t in enumerate(time[1:]):
        state_t.append(motion_model(state_t[i],u(t),R))
    z_t_GPS = [Ct_GPS@state_t[i] + multivariate_normal(np.zeros(3),Q_GPS).reshape((3,1)) for i in range(N)]
    z_t_BS = [Ht(state_t[i]) + multivariate_normal(np.zeros(4),Q_BS).reshape((4,1)) for i in range(N)]
    z_t_IMU = [state_t[i] + multivariate_normal(np.zeros(6),Q_IMU).reshape((6,1)) for i in range(N)]
    return state_t,z_t_GPS,z_t_BS,z_t_IMU 

def get_estimates_vs_time(z_t_GPS,z_t_BS,z_t_IMU,R=R,Q_GPS=Q_GPS,Q_BS=Q_BS,Q_IMU=Q_IMU):
    K_t_GPS = []
    mu_t_GPS = [mu_0]
    sigma_t_GPS = [sigma_0] 
    K_t_BS = []
    mu_t_BS = [mu_0]
    sigma_t_BS = [sigma_0]
    K_t_IMU = []
    mu_t_IMU = [mu_0]
    sigma_t_IMU = [sigma_0]

    for i,t in enumerate(time[1:]):
        mu_GPS,sigma_GPS,K_GPS = extended_kalman_filter(mu_t_GPS[i],sigma_t_GPS[i],u(t),z_t_GPS[i],R=R,C_t=Ct_GPS,Q=Q_GPS)
        mu_t_GPS.append(mu_GPS)
        sigma_t_GPS.append(sigma_GPS)
        K_t_GPS.append(K_GPS)
        mu_BS,sigma_BS,K_BS = extended_kalman_filter(mu_t_BS[i],sigma_t_BS[i],u(t),z_t_BS[i],R=R,C_t=JacHt(mu_t_BS[i]),Q=Q_BS,extended=True)
        mu_t_BS.append(mu_BS)
        sigma_t_BS.append(sigma_BS)
        K_t_BS.append(K_BS)
        mu_IMU,sigma_IMU,K_IMU = extended_kalman_filter(mu_t_IMU[i],sigma_t_IMU[i],u(t),z_t_IMU[i],R=R,C_t=Ct_IMU,Q=Q_IMU)
        mu_t_IMU.append(mu_IMU)
        sigma_t_IMU.append(sigma_IMU)
        K_t_IMU.append(K_IMU)
    estimates  = {'mu_t_GPS':mu_t_GPS,'sigma_t_GPS':sigma_t_GPS,'K_t_GPS':K_t_GPS,'mu_t_BS':mu_t_BS,'sigma_t_BS':sigma_t_BS,'K_t_BS':K_t_BS,'mu_t_IMU':mu_t_IMU,'sigma_t_IMU':sigma_t_IMU,'K_t_IMU':K_t_IMU} 
    return estimates 

def extract_estimates(est,state_t,z_t_GPS,z_t_BS,z_t_IMU):
    simulation = {'mu_t_GPS':[],'sigma_t_GPS':[],'K_t_GPS':[],'mu_t_BS':[],'sigma_t_BS':[],'K_t_BS':[],'mu_t_IMU':[],'sigma_t_IMU':[],'K_t_IMU':[]}
    
    simulation['mu_t_GPS'] = [est['mu_t_GPS'][i].reshape(6) for i in range(N)]
    simulation['sigma_t_GPS'] = [est['sigma_t_GPS'][i].reshape(6,6) for i in range(N)]
    simulation['K_t_GPS'] = [est['K_t_GPS'][i] for i in range(N-1)]
    simulation['mu_t_BS'] = [est['mu_t_BS'][i].reshape(6) for i in range(N)]
    simulation['sigma_t_BS'] = [est['sigma_t_BS'][i].reshape(6,6) for i in range(N)]
    simulation['K_t_BS'] = [est['K_t_BS'][i] for i in range(N-1)]
    simulation['mu_t_IMU'] = [est['mu_t_IMU'][i].reshape(6) for i in range(N)]
    simulation['sigma_t_IMU'] = [est['sigma_t_IMU'][i].reshape(6,6) for i in range(N)]
    simulation['K_t_IMU'] = [est['K_t_IMU'][i] for i in range(N-1)]
    simulation['state_t'] = [state_t[i].reshape(6) for i in range(N)]
    simulation['z_t_GPS'] = [z_t_GPS[i].reshape(3) for i in range(N)]
    simulation['z_t_BS'] = [z_t_BS[i].reshape(4) for i in range(N)]
    simulation['z_t_IMU'] = [z_t_IMU[i].reshape(6) for i in range(N)]
    return simulation


In [4]:
corners_of_field = [[-32,50,0],[32,50,0],[32,-50,0],[-32,-50,0]] 
corners_of_field.append(corners_of_field[0]) 
corners_of_goal = [[4,50,0],[4,50,3],[-4,50,3],[-4,50,0]] 

state_t,z_t_GPS,z_t_BS,z_t_IMU = gen_ground_truth_and_measurements(R,Q_GPS,Q_BS,Q_IMU)
estimates = get_estimates_vs_time(z_t_GPS,z_t_BS,z_t_IMU,R,Q_GPS,Q_BS,Q_IMU)
simulation = extract_estimates(estimates,state_t,z_t_GPS,z_t_BS,z_t_IMU)
state_t = simulation['state_t']


fig = go.Figure() 
fig.add_trace(go.Scatter3d(
    x =[corners_of_field[i][0] for i in range(len(corners_of_field))],
    y = [corners_of_field[i][1] for i in range(len(corners_of_field))], 
    z = [corners_of_field[i][2] for i in range(len(corners_of_field))], 
    mode = 'lines',
    name = 'Field Boundary',
    line=dict(color='red')
))
fig.add_trace(go.Scatter3d(
    x=[corners_of_goal[i][0] for i in range(len(corners_of_goal))],
    y=[corners_of_goal[i][1] for i in range(len(corners_of_goal))],
    z=[corners_of_goal[i][2] for i in range(len(corners_of_goal))], 
    mode = 'lines',
    name = 'Goal boundary',      
    line=dict(color='green')
))

fig.add_trace(go.Scatter3d(
    x=[state_t[i][0] for i in range(N)],
    y=[state_t[i][1] for i in range(N)],
    z=[state_t[i][2] for i in range(N)],
    mode='lines+markers',
    line=dict(color='blue', width=2),
    marker=dict(size=1), 
    name='Ground Truth Trajectory'
))
print("Last state_t:", state_t[-1])
fig.update_layout(
    title = "Ground Truth Trajectory of the Ball", 
    scene=dict(
        aspectmode='manual',
        aspectratio=dict(x=1, y=1, z=1),  
        zaxis=dict(range=[-30, 30]),
        xaxis = dict(range=[-60, 60]),
        yaxis= dict(range=[-60,60]),
    )
)
fig.show() 
fig.write_html(f"{plot_dir}part_a/ground_truth_trajectory.html") 

fig.add_trace(go.Scatter3d(
    x = [simulation['mu_t_GPS'][i][0] for i in range(N)],
    y = [simulation['mu_t_GPS'][i][1] for i in range(N)],
    z = [simulation['mu_t_GPS'][i][2] for i in range(N)],
    mode = 'lines+markers',
    line=dict(width=2,color='violet'), 
    marker=dict(size=1),
    name = 'GPS estimated Trajectory'
))
fig.update_layout(
    title = "GPS estimated Trajectory and Ground Truth Trajectory of the Ball",
)
fig.show() 
fig.write_html(f"{plot_dir}part_a/gps_estimated_trajectory.html")
fig.data = fig.data[:-1]

fig.add_trace(go.Scatter3d(
    x = [simulation['mu_t_BS'][i][0] for i in range(N)],
    y = [simulation['mu_t_BS'][i][1] for i in range(N)],
    z = [simulation['mu_t_BS'][i][2] for i in range(N)],
    mode = 'lines+markers',
    line=dict(width=2,color='orange'),
    marker=dict(size=1),
    name = 'BS estimated Trajectory'
))
fig.update_layout(
    title = "BS estimated Trajectory and Ground Truth Trajectory of the Ball",
)
fig.show()
fig.write_html(f"{plot_dir}part_a/bs_estimated_trajectory.html")
fig.data = fig.data[:-1] 

fig.add_trace(go.Scatter3d(
    x = [simulation['mu_t_IMU'][i][0] for i in range(N)],
    y = [simulation['mu_t_IMU'][i][1] for i in range(N)],
    z = [simulation['mu_t_IMU'][i][2] for i in range(N)],
    mode = 'lines+markers',
    line = dict(width=2,color='cyan'),
    marker = dict(size=1), 
    name='IMU estimated Trajectory',
))
fig.update_layout(
    title = "IMU estimated Trajectory and Ground Truth Trajectory of the Ball"
)
fig.show()
fig.write_html(f"{plot_dir}part_a/imu_estimated_trajectory.html") 

fig.add_trace(go.Scatter3d(
    x = [simulation['mu_t_BS'][i][0] for i in range(N)],
    y = [simulation['mu_t_BS'][i][1] for i in range(N)],
    z = [simulation['mu_t_BS'][i][2] for i in range(N)],
    mode = 'lines+markers',
    line=dict(width=2,color='orange'),
    marker=dict(size=1),
    name = 'BS estimated Trajectory'
)) 
fig.add_trace(go.Scatter3d(
    x = [simulation['mu_t_GPS'][i][0] for i in range(N)],
    y = [simulation['mu_t_GPS'][i][1] for i in range(N)],
    z = [simulation['mu_t_GPS'][i][2] for i in range(N)],
    mode = 'lines+markers',
    line=dict(width=2,color='violet'), 
    marker=dict(size=1),
    name = 'GPS estimated Trajectory'
)) 
fig.update_layout(
    title = "Estimated Trajectories and Ground Truth Trajectory of the Ball",
) 
fig.show()
fig.write_html(f"{plot_dir}part_a/estimated_trajectories.html")

Last state_t: [  3.42572567  52.7849667    3.47499174 -15.40330862  39.21570455
  -4.3827245 ]


In [5]:
# Part(c) Goal detection, Truth, sensor data, estimated trajectory 
"""
v1 + lv2 = 
y = 50 
"""
def goal_detection(trajectory):
    direction_vectors = [trajectory[i+1]-trajectory[i] for i in range(len(trajectory)-1)]
    direction_vectors.append(direction_vectors[-1]) 
    for i in range(N):
        # t*direction_vectors[i][1] = 50 - trajectory[i][1] 
        if direction_vectors[i][1]:
            t = (50 - trajectory[i][1])/direction_vectors[i][1]
            if t<0 or (t>1 and i!=N-1): # to check in case last point is before y=50 
                continue 
            x = trajectory[i][0] + t*direction_vectors[i][0]
            z = trajectory[i][2] + t*direction_vectors[i][2]
            if x>=-4 and x<=4 and z>=0 and z<=3:
                return True 
        elif trajectory[i][1] == 50:
            if trajectory[i][0]>=-4 and trajectory[i][0]<=4 and trajectory[i][2]>=0 and trajectory[i][2]<=3:
                return True 
    return False 



def compute_confidence_ellipse(mu_pos, Sigma_pos, confidence=0.95, num_points=1000):
    """
    Computes points on the confidence ellipse for a 2D Gaussian.
    
    mu_pos: 2x1 mean vector [x, z].
    Sigma_pos: 2x2 covariance matrix for x and z.
    confidence: desired confidence level (e.g. 0.95).
    num_points: number of points to sample along the ellipse.
    Returns an (num_points x 2) array of (x, z) points.
    """
    # Chi-square value for 2 degrees of freedom.
    chi2_val = stats.chi2.ppf(confidence, df=2)
    
    # Eigen-decomposition of the 2x2 covariance matrix.
    vals, vecs = np.linalg.eigh(Sigma_pos)
    order = np.argsort(vals)[::-1]
    vals = vals[order]
    vecs = vecs[:, order]
    
    theta = np.linspace(0, 2*np.pi, num_points)
    ellipse = np.zeros((num_points, 2))
    for i in range(num_points):
        # A_t point on the unit circle.
        circle_point = np.array([np.cos(theta[i]), np.sin(theta[i])])
        # Scale and rotate to get a point on the ellipse.
        # (sqrt(vals[i] * chi2_val)) gives the semi-axis lengths.
        ellipse_point = mu_pos.flatten() + vecs @ (np.sqrt(np.diag(vals * chi2_val)) @ circle_point)
        ellipse[i, :] = ellipse_point
    return ellipse 

def check_ellipse_inside_goal(ellipse, goal_bounds):
   
    x_min, x_max = goal_bounds['x_min'], goal_bounds['x_max']
    z_min, z_max = goal_bounds['z_min'], goal_bounds['z_max']
    
    # If every point satisfies the constraints, the ellipse is fully inside.
    inside = np.all((ellipse[:, 0] >= x_min) & (ellipse[:, 0] <= x_max) &
                    (ellipse[:, 1] >= z_min) & (ellipse[:, 1] <= z_max))
    return inside

def goal_detection_hypothesis_test(mu,sigma,goal_bounds={'x_min':-4,'x_max':4,'z_min':0,'z_max':3}):
    direction_vectors = [mu[i+1]-mu[i] for i in range(N-1)]
    direction_vectors.append(direction_vectors[-1]) 
    for i in range(N):
        if direction_vectors[i][1]:
            t = (50 - mu[i][1])/direction_vectors[i][1]
            if t<0 or (t>1 and i!=N-1):
                continue 
            mu_t = mu[i] + t*direction_vectors[i] 
            sigma_t = sigma[i] 
            mu_t = np.array([mu_t[0],mu_t[2]]).reshape(2,1)
            sigma_t = np.array([[sigma_t[0,0],sigma_t[0,2]],[sigma_t[2,0],sigma_t[2,2]]])
            ellipse = compute_confidence_ellipse(mu_t,sigma_t) 
            inside = check_ellipse_inside_goal(ellipse,goal_bounds)
            if inside:
                return True
        elif mu[i][1] == 50:
            mu_t = np.array([mu[i][0],mu[i][2]]).reshape(2,1)
            sigma_t = np.array([[sigma[i][0,0],sigma[i][0,2]],[sigma[i][2,0],sigma[i][2,2]]])
            ellipse = compute_confidence_ellipse(mu_t,sigma_t)
            inside = check_ellipse_inside_goal(ellipse,goal_bounds)
            if inside:
                return True
    return False
            
            
            
def goal_scoring_simulation(no_of_simulations,R=R,Q_GPS=Q_GPS,Q_BS=Q_BS,Q_IMU=Q_IMU):
    fraction_scored_ground_truth = 0
    fraction_scored_GPS = 0
    fraction_scored_IMU = 0  
    fraction_scored_GPS_estimates = 0
    fraction_scored_BS_estimates = 0
    fraction_scored_IMU_estimates = 0
    # num_GPS_FP = 0 
    # num_GPS_FN = 0 
    # num_IMU_FP = 0
    # num_IMU_FN = 0 
    for i in range(no_of_simulations):
        state_t,z_t_GPS,z_t_BS,z_t_IMU = gen_ground_truth_and_measurements(R,Q_GPS,Q_BS,Q_IMU)
        estimates = get_estimates_vs_time(z_t_GPS,z_t_BS,z_t_IMU,R,Q_GPS,Q_BS,Q_IMU)
        simulation = extract_estimates(estimates,state_t,z_t_GPS,z_t_BS,z_t_IMU)
        goal_ground_truth = goal_detection(simulation['state_t'])
        goal_GPS_measurements = goal_detection(simulation['z_t_GPS'])
        goal_IMU_measurements = goal_detection(simulation['z_t_IMU']) 
        goal_GPS_estimates = goal_detection_hypothesis_test(simulation['mu_t_GPS'],simulation['sigma_t_GPS'])
        goal_BS_estimates = goal_detection_hypothesis_test(simulation['mu_t_BS'],simulation['sigma_t_BS'])
        goal_IMU_estimates = goal_detection_hypothesis_test(simulation['mu_t_IMU'],simulation['sigma_t_IMU'])
        
        # if goal_ground_truth != goal_GPS:
        #     num_GPS_FP += (goal_ground_truth==False)
        #     num_GPS_FN += (goal_ground_truth==True)
        # if goal_ground_truth != goal_IMU:
        #     num_IMU_FP += (goal_ground_truth==False)
        #     num_IMU_FN += (goal_ground_truth==True)
            
        fraction_scored_ground_truth += goal_ground_truth
        fraction_scored_GPS += goal_GPS_measurements 
        fraction_scored_IMU += goal_IMU_measurements
        fraction_scored_GPS_estimates += goal_GPS_estimates
        fraction_scored_BS_estimates += goal_BS_estimates
        fraction_scored_IMU_estimates += goal_IMU_estimates
    
    fraction_scored_ground_truth /= no_of_simulations
    fraction_scored_GPS /= no_of_simulations
    fraction_scored_IMU /= no_of_simulations
    fraction_scored_GPS_estimates /= no_of_simulations
    fraction_scored_BS_estimates /= no_of_simulations
    fraction_scored_IMU_estimates /= no_of_simulations
    # print("GPS FP:",num_GPS_FP)
    # print("GPS FN:",num_GPS_FN)
    # print("IMU FP:",num_IMU_FP)
    # print("IMU FN:",num_IMU_FN)
    goal_simulation = {'fraction_scored_ground_truth':fraction_scored_ground_truth,'fraction_scored_GPS':fraction_scored_GPS,'fraction_scored_IMU':fraction_scored_IMU,
                        'fraction_scored_GPS_estimates':fraction_scored_GPS_estimates,'fraction_scored_BS_estimates':fraction_scored_BS_estimates,'fraction_scored_IMU_estimates':fraction_scored_IMU_estimates}
    return goal_simulation  

no_of_simulations = 1000
goal_simulation = goal_scoring_simulation(no_of_simulations)

print("Fraction of goals scored by ground truth:",goal_simulation['fraction_scored_ground_truth'])
print("Fraction of goals scored by GPS measurements:",goal_simulation['fraction_scored_GPS'])
print("Fraction of goals scored by IMU measurements:",goal_simulation['fraction_scored_IMU'])
print("Fraction of goals scored by GPS estimates:",goal_simulation['fraction_scored_GPS_estimates'])
print("Fraction of goals scored by BS estimates:",goal_simulation['fraction_scored_BS_estimates'])
print("Fraction of goals scored by IMU estimates:",goal_simulation['fraction_scored_IMU_estimates'])


        


Fraction of goals scored by ground truth: 0.291
Fraction of goals scored by GPS measurements: 0.291
Fraction of goals scored by IMU measurements: 0.289
Fraction of goals scored by GPS estimates: 0.247
Fraction of goals scored by BS estimates: 0.221
Fraction of goals scored by IMU estimates: 0.255


In [6]:
# Part (d) varying the noise levels 

number_of_simulations = 200
sig_pos_ranges = [0.0]+[0.01*2**i for i in range(0,5)]
sig_vel_ranges = [0.0]+[0.1*2**i for i in range(0,5)]
sig_sensor_ranges = [0.01*2**i for i in range(0,5)] 

def goal_scoring_simulation_varying_noise_GPS(sig_pos_ranges,sig_vel_ranges,sig_sensor_ranges,number_of_simulations,R=R,Q_GPS=Q_GPS,Q_BS=Q_BS,Q_IMU=Q_IMU):
    simulation_table = {'sig_pos':[],'sig_vel':[],'sig_GPS':[],
                        'Ground Truth':[],'GPS measurement':[],'IMU measurement':[],
                        'BS estimates':[],'IMU estimates':[],'GPS estimates':[],
                        'no of simulations':number_of_simulations
                        
                        }
    for sigp in sig_pos_ranges:
        for sigv in sig_vel_ranges: 
            for sigs in sig_sensor_ranges: 
                R = np.diag([sigp**2,sigp**2,sigp**2,sigv**2,sigv**2,sigv**2]) 
                Q_GPS = sigs**2*np.eye(3)
                goal_simulation = goal_scoring_simulation(number_of_simulations,R,Q_GPS,Q_BS,Q_IMU) 
                simulation_table['sig_pos'].append(sigp)
                simulation_table['sig_vel'].append(sigv)
                simulation_table['sig_GPS'].append(sigs)
                simulation_table['Ground Truth'].append(goal_simulation['fraction_scored_ground_truth'])
                simulation_table['GPS measurement'].append(goal_simulation['fraction_scored_GPS'])
                simulation_table['IMU measurement'].append(goal_simulation['fraction_scored_IMU'])
                simulation_table['BS estimates'].append(goal_simulation['fraction_scored_BS_estimates'])
                simulation_table['IMU estimates'].append(goal_simulation['fraction_scored_IMU_estimates'])
                simulation_table['GPS estimates'].append(goal_simulation['fraction_scored_GPS_estimates'])
    return simulation_table
                 
                    
simulation_table = goal_scoring_simulation_varying_noise_GPS(sig_pos_ranges,sig_vel_ranges,sig_sensor_ranges,number_of_simulations,R,Q_GPS,Q_BS,Q_IMU)          
print(simulation_table)         

{'sig_pos': [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.01, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.02, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.04, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.08, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.16, 0.1

In [7]:
simulation_table_df = pd.DataFrame(simulation_table)
simulation_table_df.to_csv("simulation_table.csv")

In [8]:
# Part (e) varying position noise and GPS sensor noise, and analysing 2D projection 
sig_pos_ranges = [0.0,0.01,0.1,1.0]
sig_sensor_ranges = [0.01,0.05,0.1,1.0]
def vary_pos_GPS_noise_and_2D_proj(sig_pos_ranges,sig_sensor_ranges,R=R,Q_GPS=Q_GPS,Q_BS=Q_BS,Q_IMU=Q_IMU):
    fig = go.Figure() 
    for sigp in sig_pos_ranges:
        for sigs in sig_sensor_ranges: 
            R = np.diag([sigp**2,sigp**2,sigp**2,sigvx**2,sigvy**2,sigvz**2]) 
            Q_GPS = sigs**2*np.eye(3) 
            state_t,z_t_GPS,z_t_BS,z_t_IMU = gen_ground_truth_and_measurements(R,Q_GPS,Q_BS,Q_IMU) 
            estimates = get_estimates_vs_time(z_t_GPS,z_t_BS,z_t_IMU,R,Q_GPS,Q_BS,Q_IMU) 
            simulation = extract_estimates(estimates,state_t,z_t_GPS,z_t_BS,z_t_IMU) 
            # Extract and plot the XY projections for ground truth, GPS sensor measurements, and estimated GPS trajectory

            # Ground truth XY coordinates
            truth_x = [s[0] for s in simulation['state_t']]
            truth_y = [s[1] for s in simulation['state_t']]

            # GPS sensor measurements XY coordinates
            gps_x = [z[0] for z in simulation['z_t_GPS']]
            gps_y = [z[1] for z in simulation['z_t_GPS']]

            # Estimated GPS trajectory XY coordinates
            est_x = [mu[0] for mu in simulation['mu_t_GPS']]
            est_y = [mu[1] for mu in simulation['mu_t_GPS']]

            # Add traces for the trajectories
            fig.add_trace(go.Scatter(
                x=truth_x, y=truth_y,
                mode='lines+markers',
                name=f'Ground Truth Trajectory (XY), sig_pos={sigp}, sig_GPS={sigs}',
                line=dict(color='blue')
            ))
            fig.add_trace(go.Scatter(
                x=gps_x, y=gps_y,
                mode='lines+markers',
                name=f'GPS Measurements (XY), sig_pos={sigp}, sig_GPS={sigs}',
                marker=dict(color='red', size=4, symbol='circle')
            ))
            fig.add_trace(go.Scatter(
                x=est_x, y=est_y,
                mode='lines+markers',
                name=f'GPS Estimated Trajectory (XY), sig_pos={sigp}, sig_GPS={sigs}',
                line=dict(color='green')
            ))

            # Plot uncertainty ellipses (one standard deviation) for the estimated trajectory
            # To avoid clutter, plot ellipses for every 10th point
            for i in range(0, len(simulation['mu_t_GPS']), 10):
                mu = simulation['mu_t_GPS'][i]
                sigma = simulation['sigma_t_GPS'][i]
                # Extract the XY components: mean and covariance
                mu_xy = mu[:2]
                sigma_xy = sigma[:2, :2]
                # Generate points on the unit circle
                t = np.linspace(0, 2*np.pi, 100)
                circle = np.array([np.cos(t), np.sin(t)])  # shape (2, 100)
                # Eigen-decomposition of the covariance matrix
                eigenvals, eigenvecs = np.linalg.eigh(sigma_xy)
                # Scale by standard deviations (one sigma)
                ellipse = mu_xy.reshape(2,1) + eigenvecs @ np.diag(np.sqrt(eigenvals)) @ circle
                fig.add_trace(go.Scatter(
                    x=ellipse[0], y=ellipse[1],
                    mode='lines',
                    line=dict(color='black', dash='dash'),
                    showlegend=False
                ))

            fig.update_layout(title='XY Projection with Uncertainty Ellipses (One Std Dev)',xaxis_title='X',yaxis_title='Y')
            # fig.update_layout(yaxis=dict(autorange="reversed"))  
            fig.update_layout(xaxis=dict(autorange="reversed"))
            fig.show()
            fig = go.Figure()
            

vary_pos_GPS_noise_and_2D_proj(sig_pos_ranges,sig_sensor_ranges,R,Q_GPS,Q_BS,Q_IMU)
            

In [9]:


# ----------------------------------------------------------------------------
# Comments on the orientation of the ellipses and the lengths of their axes:
#
#  • Notice that the uncertainty ellipses (drawn from the 2x2 position covariance)
#    are not oriented arbitrarily. Their major (and minor) axes “point” in the
#    directions where the EKF is less (or more) confident. When the measurement is
#    a distance (range) measurement from a fixed BS, the measurement provides
#    high information along the radial direction from the BS but almost no direct
#    information in the tangential direction. As a result, the uncertainty ellipses
#    tend to be elongated tangent to the line from the BS to the ball.
#
#  • In the case of two base stations (B3B4), the combined geometry gives measurements
#    from two distinct locations. This typically reduces uncertainty in the direction
#    perpendicular to the line joining the BSs and produces a more “rounded” ellipse.
#
#  • The plotted lengths of the major and minor axes represent (approximately) the 2-sigma
#    spread of the position uncertainty in the two principal directions. They indicate
#    the level of confidence and the “shape” of the uncertainty in the state estimate;
#    the larger the major axis, the greater the ambiguity in that particular direction.
#
#  • Comparing the four cases: the ellipses from the single BS cases are highly
#    anisotropic (elongated in the tangential direction relative to that BS), while the
#    measurements coming from the paired BS (B3B4) tend to yield both a smaller major
#    axis and a more balanced minor axis since the geometry “observes” the target from
#    two sides.
#
# These trends explain why a sensor that provides only range measurements may lead to
# large uncertainty in one direction, and why sensor fusion (or multiple sensors from
# different viewpoints) can greatly reduce ambiguity.
#
# ─────────────────────────────────────────────────────────────────────────────
#


In [10]:
# Part (f) New Simulation with 2D Trajectory and Uncertainty Ellipses 

time_step = 0.01         
N = 250   
time = [i*time_step for i in range(N)]


mu_0 = np.array([0.0, -50.0, 0.0, 40.0]).reshape(4,1)
sigma_0 = np.eye(4)


sig_pos = 0.01
sig_vel = 0.1
R_proc = np.diag([sig_pos**2, sig_pos**2, sig_vel**2, sig_vel**2])


A_t = np.array([[1, 0, time_step,  0],
              [0, 1,  0, time_step],
              [0, 0,  1,  0],
              [0, 0,  0,  1]])


B1 = np.array([-32, 50])
B2 = np.array([-32, -50])
B3 = np.array([-32, 0])
B4 = np.array([32, 0])

# Define measurement functions and their Jacobians.
def Ht(state, BS):
    # state is 4x1; use only position components
    pos = state[0:2, 0]
    return np.array([np.linalg.norm(pos - BS)])

def meas_jacobian(state, BS):
    pos = state[0:2, 0]
    diff = pos - BS
    d = np.linalg.norm(diff)
    # Avoid division by zero:
    if d < 1e-8:
        d = 1e-8
    H = np.zeros((1,4))
    # Derivative w.r.t x and y; no dependence on velocity:
    H[0,0] = diff[0] / d
    H[0,1] = diff[1] / d
    return H

# For the combined measurement from two base stations:
def meas_func_combined(state):
    return np.vstack([Ht(state, B3), Ht(state, B4)])

def meas_jacobian_combined(state):
    H1 = meas_jacobian(state, B3)
    H2 = meas_jacobian(state, B4)
    return np.vstack([H1, H2])

# BS measurement noise:
sig_BS = 0.1

# Measurement noise covariance for a single BS and for two measurements:
Q_single = np.array([[sig_BS**2]])
Q_combined = np.diag([sig_BS**2, sig_BS**2])

# Define EKF update function.
def ekf_update(mu_pred, Sigma_pred, z, h_func, H_func, Q):
    # compute predicted measurement:
    z_pred = h_func(mu_pred)
    # innovation:
    y = z - z_pred
    # Jacobian at mu_pred:
    H = H_func(mu_pred)
    S = H @ Sigma_pred @ H.T + Q
    K = Sigma_pred @ H.T @ np.linalg.inv(S)
    mu_upd = mu_pred + K @ y
    Sigma_upd = (np.eye(len(mu_pred)) - K @ H) @ Sigma_pred
    return mu_upd, Sigma_upd

# Function to simulate one trajectory (ground truth) using the process model:
def simulate_truth(mu_0, A_t, R_proc, N):
    state = mu_0.copy()
    truth = [state]
    for k in range(1, N):
        noise = np.random.multivariate_normal(np.zeros(4), R_proc).reshape(4,1)
        state = A_t @ state + noise
        truth.append(state)
    return truth

# EKF simulation for a given measurement model (h_func, H_func, Q)
def run_ekf(truth, A_t, R_proc, h_func, H_func, Q):
    n = len(truth)
    mu = truth[0].copy()   # start with true initial state
    Sigma = sigma_0.copy()
    estimates = [mu]
    covariances = [Sigma]
    for k in range(1, n):
        # Prediction
        mu_pred = A_t @ mu
        Sigma_pred = A_t @ Sigma @ A_t.T + R_proc
        # Generate measurement from truth:
        z_true = h_func(truth[k])
        meas_noise = np.random.multivariate_normal(np.zeros(Q.shape[0]), Q).reshape(Q.shape[0],1)
        z = z_true + meas_noise
        # Update:
        mu, Sigma = ekf_update(mu_pred, Sigma_pred, z, h_func, H_func, Q)
        estimates.append(mu)
        covariances.append(Sigma)
    return estimates, covariances

# Function to compute ellipse parameters from a 2x2 covariance matrix.
def get_ellipse_params(Sigma_pos, nsig=1):
    # Eigen decomposition of 2x2 covariance Sigma_pos:
    eigvals, eigvecs = np.linalg.eig(Sigma_pos)
    # Sort eigenvalues in descending order:
    order = np.argsort(eigvals)[::-1]
    eigvals = eigvals[order]
    eigvecs = eigvecs[:, order]
    # The lengths of the semi-axes:
    a = nsig * np.sqrt(eigvals[0])
    b = nsig * np.sqrt(eigvals[1])
    # Orientation angle of the ellipse (in degrees)
    angle = np.degrees(np.arctan2(eigvecs[1,0], eigvecs[0,0]))
    return a, b, angle

# Helper function to generate ellipse (x,y) points for plotting.
def ellipse_points(cx, cy, a, b, angle_deg, num_points=100):
    """
    Compute x and y points for an ellipse.
    
    Parameters:
        cx, cy: Center of the ellipse.
        a, b: Semi-major and semi-minor axes.
        angle_deg: Rotation angle of the ellipse (in degrees).
        num_points: Number of points to generate along the ellipse.
    """
    t = np.linspace(0, 2*np.pi, num_points)
    angle_rad = np.deg2rad(angle_deg)
    x = cx + a * np.cos(t) * np.cos(angle_rad) - b * np.sin(t) * np.sin(angle_rad)
    y = cy + a * np.cos(t) * np.sin(angle_rad) + b * np.sin(t) * np.cos(angle_rad)
    return x, y

# Define the four measurement cases:
cases = {
    "B1": {"h_func": lambda state: Ht(state, B1),
           "H_func": lambda state: meas_jacobian(state, B1),
           "Q": Q_single,
           "BS_list": [B1]},
    "B2": {"h_func": lambda state: Ht(state, B2),
           "H_func": lambda state: meas_jacobian(state, B2),
           "Q": Q_single,
           "BS_list": [B2]},
    "B3": {"h_func": lambda state: Ht(state, B3),
           "H_func": lambda state: meas_jacobian(state, B3),
           "Q": Q_single,
           "BS_list": [B3]},
    "B3B4": {"h_func": meas_func_combined,
             "H_func": meas_jacobian_combined,
             "Q": Q_combined,
             "BS_list": [B3, B4]}
}

# Simulate the ground truth trajectory:
truth = simulate_truth(mu_0, A_t, R_proc, N)

# Run the EKF for each case:
results = {}
for key, meas_data in cases.items():
    est, cov = run_ekf(truth, A_t, R_proc, meas_data["h_func"], meas_data["H_func"], meas_data["Q"])
    results[key] = {"estimates": est, "covariances": cov}

# --------------------------------------------------------------------
# Plotting using Plotly for each case.
# We create a two-panel figure:
#   - Left: XY trajectory (truth in blue, EKF estimate in green) with uncertainty ellipses and BS "rays".
#   - Right: Evolution of the uncertainty ellipse's major and minor axes over time.

for key, meas_data in cases.items():
    est = results[key]["estimates"]
    cov = results[key]["covariances"]

    # Prepare arrays for positions:
    truth_xy = np.array([s[0:2,0] for s in truth])
    est_xy = np.array([s[0:2,0] for s in est])
    
    # Store the full (2-sigma) lengths of the ellipse axes:
    major_axis = []
    minor_axis = []
    
    # Create subplots: one row, two columns.
    fig = sp.make_subplots(rows=1, cols=2,
                           subplot_titles=(f"Case {key}: XY Trajectory with Uncertainty Ellipses",
                                           f"Case {key}: Evolution of Ellipse Axes Lengths"))
    
    # Left Panel: Plot the truth and EKF trajectories.
    fig.add_trace(go.Scatter(x=truth_xy[:,0],
                             y=truth_xy[:,1],
                             mode='lines',
                             name='Truth',
                             line=dict(color='blue')),
                  row=1, col=1)
    
    fig.add_trace(go.Scatter(x=est_xy[:,0],
                             y=est_xy[:,1],
                             mode='lines',
                             name='EKF Estimate',
                             line=dict(color='green', dash='dash')),
                  row=1, col=1)
    
    # Plot BS positions as red markers.
    for idx, BS in enumerate(meas_data["BS_list"]):
        name = "BS" if idx == 0 else ""
        fig.add_trace(go.Scatter(x=[BS[0]],
                                 y=[BS[1]],
                                 mode='markers',
                                 marker=dict(color='red', size=10),
                                 name=name),
                      row=1, col=1)
    
    # Draw lines ("rays") from each BS to the estimated position every 20 time steps.
    for i in range(0, N, 20):
        pos = est_xy[i]
        for BS in meas_data["BS_list"]:
            fig.add_trace(go.Scatter(x=[BS[0], pos[0]],
                                     y=[BS[1], pos[1]],
                                     mode='lines',
                                     line=dict(color='black', dash='dot', width=1),
                                     showlegend=False),
                          row=1, col=1)
    
    # Plot uncertainty ellipses every 10 time steps.
    for i in range(0, N, 10):
        Sigma_pos = cov[i][0:2,0:2]
        a_len, b_len, angle = get_ellipse_params(Sigma_pos, nsig=1)
        major_axis.append(2 * a_len)  # full length (2-sigma)
        minor_axis.append(2 * b_len)
        cx, cy = est_xy[i,0], est_xy[i,1]
        x_ellipse, y_ellipse = ellipse_points(cx, cy, a_len, b_len, angle, num_points=100)
        fig.add_trace(go.Scatter(x=x_ellipse,
                                 y=y_ellipse,
                                 mode='lines',
                                 line=dict(color='magenta', width=1.5),
                                 showlegend=False),
                      row=1, col=1)
    
    # Update left panel axes labels.
    fig.update_xaxes(title_text="X", row=1, col=1)
    fig.update_yaxes(title_text="Y", row=1, col=1)
    
    # Right Panel: Plot the evolution of the major and minor axis lengths.
    time_axis = time[:len(major_axis)]
    fig.add_trace(go.Scatter(x=time_axis,
                             y=major_axis,
                             mode='lines',
                             name='Major Axis',
                             line=dict(color='red')),
                  row=1, col=2)
    fig.add_trace(go.Scatter(x=time_axis,
                             y=minor_axis,
                             mode='lines',
                             name='Minor Axis',
                             line=dict(color='blue')),
                  row=1, col=2)
    
    fig.update_xaxes(title_text="Time (s)", row=1, col=2)
    fig.update_yaxes(title_text="Axis length (2-sigma)", row=1, col=2)
    
    fig.update_layout(title_text=f"Case {key}",
                      legend=dict(x=0.02, y=0.98),
                      width=1000, height=500)
    
    fig.show()
